In [4]:
import os, shutil, xml, zipfile, pandas, glob
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import xmltodict 

sample_hwp_file = r'../data/template1.hwpx'
temp_dir = r'../temp'

def extract_lxml(hwpx_file):
    temp_base_name = os.path.join(temp_dir, os.path.splitext(os.path.basename(hwpx_file))[0])
    temp_zip_file = f'{temp_base_name}.zip'
    target_xml_file = os.path.join(temp_base_name, 'Contents', 'section0.xml')

    if not os.path.exists(temp_zip_file):
        shutil.copy(hwpx_file, temp_zip_file)

    with zipfile.ZipFile(temp_zip_file, 'r') as zf:
        zf.extractall(path=temp_base_name)

    with open(target_xml_file, 'r') as file:
        xml_doc = file.read()

    return hwpx_file, BeautifulSoup(xml_doc, 'lxml')

soup = extract_lxml(sample_hwp_file)

/Users/sikso/workspace/Miscellaneous/miscellaneous-export-hwp/.venv/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [6]:
arr_soup = [extract_lxml(hwpx_file) for hwpx_file in glob.glob(r'../data/*.hwpx')]

In [7]:
_info_dict = {
    '보고일': None,
    '보고자': None,
    '평가일': None,
    '평가자': None,
    '병록번호': None,
    '환자명': None,
    '나이': None,
    '성별': None,
    '유해사례 내용': None,
    '유해사례 보고 내용': None,
    '약사평가': None,
    '참고문헌': None
}
_score_dict ={
    'WHO-UMC causality assessment': None,
    'Naranjo Scale': None,
    '한국형 Algorithm Ver 2.0': None
}

In [14]:
# ../data/template_2016.hwpx

def remove_special_characters(text):
    pattern = r'[^a-zA-Z0-9가-힣]'
    return re.sub(pattern, '', text)

def preprocess_tag_string(tag):
    _preprocessing_tag_string = re.sub(r'^\s*:\s*', '', tag.string)
    _preprocessing_tag_string = re.sub(r'^\s*-\s*', '', _preprocessing_tag_string)
    return _preprocessing_tag_string.strip()

def extract_info(arr_tag_string):
    _dict = {}
    keys = _info_dict.keys()
    length = len(arr_tag_string)
    i = 0
    value = ''
    
    while i < length:
        try:
            if arr_tag_string[i] in keys:
                target_key = arr_tag_string[i]
                value = ''
            
            elif arr_tag_string[i] == '담당의사 등 전문가 의견' or arr_tag_string[i] == '환자정보':
                pass
            
            elif arr_tag_string[i] == 'WHO-UMC causality assessment':
                _dict['WHO-UMC causality assessment'] = arr_tag_string[i+1]
                
            
            elif arr_tag_string[i] == 'Naranjo Scale':
                _dict['Naranjo Scale'] = arr_tag_string[i+3]
                _dict['Naranjo Scale (Total Score)'] = arr_tag_string[i+2]
                    
            
            elif arr_tag_string[i] == '한국형 Algorithm Ver 2.0':
                _dict['한국형 Algorithm Ver 2.0'] = arr_tag_string[i+3]
                _dict['한국형 Algorithm Ver 2.0 (Total Score)'] = arr_tag_string[i+2]
            
            else:
                value += arr_tag_string[i]
                
            if i == length -1 or arr_tag_string[i+1] in keys:
                _dict[target_key] = value
                
        except:
            pass
                
        i += 1
    del _dict['참고문헌']
    return _dict
            
keys = _info_dict.keys()

soup = arr_soup[0]
print(soup[0])
tags = [preprocess_tag_string(tag.string) for tag in soup[1].select('hp\:t') if tag.string]

extract_info(tags)

../data/template_2016.hwpx


{'보고일': '2001-01-05',
 '보고자': '김길동',
 '평가일': '2001-01-08',
 '평가자': '강길동',
 '병록번호': '11111110',
 '환자명': '강길길',
 '나이': '31',
 '성별': '여자',
 '유해사례 내용': '국가는 사회보장·사회복지의 증진에 노력할 의무를 진다.',
 '약사평가': '모든 국민은 헌법과 법률이 정한 법관에 의하여 법률에 의한 재판을 받을 권리를 가진다. 군인은 현역을 면한 후가 아니면 국무위원으로 임명될 수 없다. 국민경제자문회의의 조직·직무범위 기타 필요한 사항은 법률로 정한다. 선거운동은 각급 선거관리위원회의 관리하에 법률이 정하는 범위안에서 하되, 균등한 기회가 보장되어야 한다. 대통령·국무총리·국무위원·행정각부의 장·헌법재판소 재판관·법관·중앙선거관리위원회 위원·감사원장·감사위원 기타 법률이 정한 공무원이 그 직무집행에 있어서 헌법이나 법률을 위배한 때에는 국회는 탄핵의 소추를 의결할 수 있다. 모든 국민은 근로의 의무를 진다. 국가는 근로의 의무의 내용과 조건을 민주주의원칙에 따라 법률로 정한다.대한민국은 국제평화의 유지에 노력하고 침략적 전쟁을 부인한다. 대통령후보자가 1인일 때에는 그 득표수가 선거권자 총수의 3분의 1 이상이 아니면 대통령으로 당선될 수 없다. 대통령은 내란 또는 외환의 죄를 범한 경우를 제외하고는 재직중 형사상의 소추를 받지 아니한다. 행정권은 대통령을 수반으로 하는 정부에 속한다. 국가원로자문회의의 의장은 직전대통령이 된다. 다만, 직전대통령이 없을 때에는 대통령이 지명한다. 공무원의 신분과 정치적 중립성은 법률이 정하는 바에 의하여 보장된다. 군인·군무원·경찰공무원 기타 법률이 정하는 자가 전투·훈련등 직무집행과 관련하여 받은 손해에 대하여는 법률이 정하는 보상외에 국가 또는 공공단체에 공무원의 직무상 불법행위로 인한 배상은 청구할 수 없다.',
 'WHO-UMC causality assessment': '가능함(possible)',
 '한국형 Alg

In [24]:
# ../data/template_2015_3.hwpx

def remove_special_characters(text):
    pattern = r'[^a-zA-Z0-9가-힣]'
    return re.sub(pattern, '', text)

def preprocess_tag_string(tag):
    _preprocessing_tag_string = re.sub(r'^\s*:\s*', '', tag.string)
    _preprocessing_tag_string = re.sub(r'^\s*-\s*', '', _preprocessing_tag_string)
    return _preprocessing_tag_string.strip()

def extract_info(arr_tag_string):
    _dict = {}
    keys = _info_dict.keys()
    length = len(arr_tag_string)
    i = 0
    value = ''
    while i < length:
        # print(arr_tag_string[i])
        try:
            if arr_tag_string[i] in keys:
                target_key = arr_tag_string[i]
                value = ''
            
            elif arr_tag_string[i] == '담당의사 등 전문가 의견' or arr_tag_string[i] == '환자정보':
                pass
            
            elif arr_tag_string[i] == 'Naranjo알고리즘':
                _dict['WHO-UMC causality assessment'] = arr_tag_string[i+1]
                _dict['Naranjo Scale'] = arr_tag_string[i+2]
                _dict['한국형 Algorithm Ver 2.0'] = arr_tag_string[i+3]
                
            
            elif arr_tag_string[i] == 'Naranjo Scale':
                _dict['Naranjo Scale (Total Score)'] = arr_tag_string[i+6]
                    
            
            elif arr_tag_string[i] == '한국형 Algorithm Ver 2.0':
                _dict['한국형 Algorithm Ver 2.0 (Total Score)'] = arr_tag_string[i+2]
            
            else:
                value += arr_tag_string[i]
                
            if i == length -1 or arr_tag_string[i+1] in keys:
                _dict[target_key] = value
                
        except:
            pass
                
        i += 1
    del _dict['참고문헌']
    return _dict
            
keys = _info_dict.keys()

soup = arr_soup[2]
print(soup[0])
tags = [preprocess_tag_string(tag.string) for tag in soup[1].select('hp\:t') if tag.string]

extract_info(tags)

../data/template_2015_3.hwpx


{'보고일': '2001-01-01',
 '보고자': '강길동',
 '평가일': '2001-01-30',
 '평가자': '정길동',
 '병록번호': '10011001',
 '환자명': '김길동',
 '나이': '45',
 '성별': '남자',
 '유해사례 내용': '(위장관)오심/울렁거림',
 '약사평가': '각급 선거관리위원회는 선거인명부의 작성등 선거사무와 국민투표사무에 관하여 관계 행정기관에 필요한 지시를 할 수 있다. 국가는 지역간의 균형있는 발전을 위하여 지역경제를 육성할 의무를 진다. 대통령·국무총리·국무위원·행정각부의 장·헌법재판소 재판관·법관·중앙선거관리위원회 위원·감사원장·감사위원 기타 법률이 정한 공무원이 그 직무집행에 있어서 헌법이나 법률을 위배한 때에는 국회는 탄핵의 소추를 의결할 수 있다. 국가는 전통문화의 계승·발전과 민족문화의 창달에 노력하여야 한다. 국가는 법률이 정하는 바에 의하여 재외국민을 보호할 의무를 진다. 헌법재판소의 조직과 운영 기타 필요한 사항은 법률로 정한다.',
 'WHO-UMC causality assessment': 'probable',
 'Naranjo Scale': 'doubtful',
 '한국형 Algorithm Ver 2.0': 'possible',
 'Naranjo Scale (Total Score)': '4',
 '한국형 Algorithm Ver 2.0 (Total Score)': '9'}

In [25]:
# ../data/template_2015_2.hwpx

def remove_special_characters(text):
    pattern = r'[^a-zA-Z0-9가-힣]'
    return re.sub(pattern, '', text)

def preprocess_tag_string(tag):
    _preprocessing_tag_string = re.sub(r'^\s*:\s*', '', tag.string)
    _preprocessing_tag_string = re.sub(r'^\s*-\s*', '', _preprocessing_tag_string)
    return _preprocessing_tag_string.strip()

def extract_info(arr_tag_string):
    _dict = {}
    keys = _info_dict.keys()
    length = len(arr_tag_string)
    i = 0
    value = ''
    while i < length:
        # print(arr_tag_string[i])
        try:
            if arr_tag_string[i] in keys:
                target_key = arr_tag_string[i]
                value = ''
            
            elif arr_tag_string[i] == '담당의사 등 전문가 의견' or arr_tag_string[i] == '환자정보':
                pass
            
            elif arr_tag_string[i] == 'Naranjo알고리즘':
                _dict['WHO-UMC causality assessment'] = arr_tag_string[i+1]
                _dict['Naranjo Scale'] = arr_tag_string[i+2]
                _dict['한국형 Algorithm Ver 2.0'] = arr_tag_string[i+3]
                
            
            elif arr_tag_string[i] == 'Naranjo Scale':
                _dict['Naranjo Scale (Total Score)'] = arr_tag_string[i+6]
                    
            
            elif arr_tag_string[i] == '한국형 Algorithm Ver 2.0':
                _dict['한국형 Algorithm Ver 2.0 (Total Score)'] = arr_tag_string[i+2]
            
            else:
                value += arr_tag_string[i]
                
            if i == length -1 or arr_tag_string[i+1] in keys:
                _dict[target_key] = value
                
        except:
            pass
                
        i += 1
    del _dict['참고문헌']
    return _dict
            
keys = _info_dict.keys()

soup = arr_soup[3]
print(soup[0])
tags = [preprocess_tag_string(tag.string) for tag in soup[1].select('hp\:t') if tag.string]

extract_info(tags)

../data/template_2015_2.hwpx


{'보고일': '2001-01-01',
 '보고자': '강길동',
 '평가일': '2001-01-30',
 '평가자': '정길동',
 '병록번호': '10011001',
 '환자명': '김길동',
 '나이': '45',
 '성별': '남자',
 '유해사례 내용': '(위장관)오심/울렁거림',
 '약사평가': '각급 선거관리위원회는 선거인명부의 작성등 선거사무와 국민투표사무에 관하여 관계 행정기관에 필요한 지시를 할 수 있다. 국가는 지역간의 균형있는 발전을 위하여 지역경제를 육성할 의무를 진다. 대통령·국무총리·국무위원·행정각부의 장·헌법재판소 재판관·법관·중앙선거관리위원회 위원·감사원장·감사위원 기타 법률이 정한 공무원이 그 직무집행에 있어서 헌법이나 법률을 위배한 때에는 국회는 탄핵의 소추를 의결할 수 있다. 국가는 전통문화의 계승·발전과 민족문화의 창달에 노력하여야 한다. 국가는 법률이 정하는 바에 의하여 재외국민을 보호할 의무를 진다. 헌법재판소의 조직과 운영 기타 필요한 사항은 법률로 정한다.',
 'WHO-UMC causality assessment': 'probable',
 'Naranjo Scale': 'doubtful',
 '한국형 Algorithm Ver 2.0': 'possible',
 'Naranjo Scale (Total Score)': '4',
 '한국형 Algorithm Ver 2.0 (Total Score)': '9'}

In [29]:
# ../data/template_2015.hwpx

def remove_special_characters(text):
    pattern = r'[^a-zA-Z0-9가-힣]'
    return re.sub(pattern, '', text)

def preprocess_tag_string(tag):
    _preprocessing_tag_string = re.sub(r'^\s*:\s*', '', tag.string)
    _preprocessing_tag_string = re.sub(r'^\s*-\s*', '', _preprocessing_tag_string)
    return _preprocessing_tag_string.strip()

def extract_info(arr_tag_string):
    _dict = {}
    keys = _info_dict.keys()
    length = len(arr_tag_string)
    i = 0
    value = ''
    while i < length:
        # print(arr_tag_string[i])
        try:
            if arr_tag_string[i] in keys:
                target_key = arr_tag_string[i]
                value = ''
            
            elif arr_tag_string[i] == '담당의사 등 전문가 의견' or arr_tag_string[i] == '환자정보':
                pass
            
            elif arr_tag_string[i] == 'Naranjo알고리즘':
                _dict['WHO-UMC causality assessment'] = arr_tag_string[i+1]
                _dict['Naranjo Scale'] = arr_tag_string[i+2]
                _dict['한국형 Algorithm Ver 2.0'] = arr_tag_string[i+3]
                
            
            elif arr_tag_string[i] == 'Naranjo Scale':
                _dict['Naranjo Scale (Total Score)'] = arr_tag_string[i+6]
                    
            
            elif arr_tag_string[i] == '한국형 Algorithm Ver 2.0':
                _dict['한국형 Algorithm Ver 2.0 (Total Score)'] = arr_tag_string[i+2]
            
            else:
                value += arr_tag_string[i]
                
            if i == length -1 or arr_tag_string[i+1] in keys:
                _dict[target_key] = value
                
        except:
            pass
                
        i += 1
    del _dict['참고문헌']
    return _dict
            
keys = _info_dict.keys()

soup = arr_soup[4]
print(soup[0])
tags = [preprocess_tag_string(tag.string) for tag in soup[1].select('hp\:t') if tag.string]

extract_info(tags)

../data/template_2015.hwpx


{'보고일': '2001-01-01',
 '보고자': '강길동',
 '평가일': '2001-01-30',
 '평가자': '정길동',
 '병록번호': '10011001',
 '환자명': '김길동',
 '나이': '45',
 '성별': '남자',
 '유해사례 내용': '(위장관)오심/울렁거림',
 '약사평가': '각급 선거관리위원회는 선거인명부의 작성등 선거사무와 국민투표사무에 관하여 관계 행정기관에 필요한 지시를 할 수 있다. 국가는 지역간의 균형있는 발전을 위하여 지역경제를 육성할 의무를 진다. 대통령·국무총리·국무위원·행정각부의 장·헌법재판소 재판관·법관·중앙선거관리위원회 위원·감사원장·감사위원 기타 법률이 정한 공무원이 그 직무집행에 있어서 헌법이나 법률을 위배한 때에는 국회는 탄핵의 소추를 의결할 수 있다. 국가는 전통문화의 계승·발전과 민족문화의 창달에 노력하여야 한다. 국가는 법률이 정하는 바에 의하여 재외국민을 보호할 의무를 진다. 헌법재판소의 조직과 운영 기타 필요한 사항은 법률로 정한다.',
 'WHO-UMC causality assessment': 'probable',
 'Naranjo Scale': 'doubtful',
 '한국형 Algorithm Ver 2.0': 'possible',
 'Naranjo Scale (Total Score)': '4',
 '한국형 Algorithm Ver 2.0 (Total Score)': '9'}

In [46]:
# ../data/template_2018.hwpx

def remove_special_characters(text):
    pattern = r'[^a-zA-Z0-9가-힣]'
    return re.sub(pattern, '', text)

def preprocess_tag_string(tag):
    _preprocessing_tag_string = re.sub(r'^\s*:\s*', '', tag.string)
    _preprocessing_tag_string = re.sub(r'^\s*-\s*', '', _preprocessing_tag_string)
    return _preprocessing_tag_string.strip()

def extract_info(arr_tag_string):
    _dict = {}
    keys = _info_dict.keys()
    length = len(arr_tag_string)
    i = 0
    value = ''
    while i < length:
        # print(arr_tag_string[i])
        try:
            if arr_tag_string[i] in keys:
                target_key = arr_tag_string[i]
                value = ''
            
            elif arr_tag_string[i] == '담당의사 등 전문가 의견' or arr_tag_string[i] == '환자정보':
                pass
            
            elif arr_tag_string[i] == 'Naranjo알고리즘':
                _dict['WHO-UMC causality assessment'] = arr_tag_string[i+1]
                _dict['Naranjo Scale'] = arr_tag_string[i+2]
                _dict['한국형 Algorithm Ver 2.0'] = arr_tag_string[i+3]
                
            
            elif arr_tag_string[i] == 'Naranjo Scale':
                _dict['Naranjo Scale (Total Score)'] = arr_tag_string[i+2]
                    
            
            elif arr_tag_string[i] == '한국형 Algorithm Ver 2.0':
                _dict['한국형 Algorithm Ver 2.0 (Total Score)'] = arr_tag_string[i+2]
            
            else:
                value += arr_tag_string[i]
                
            if i == length -1 or arr_tag_string[i+1] in keys:
                _dict[target_key] = value
                
        except:
            pass
                
        i += 1
    del _dict['참고문헌']
    return _dict
            
keys = _info_dict.keys()

soup = arr_soup[5]
print(soup[0])
tags = [preprocess_tag_string(tag.string) for tag in soup[1].select('hp\:t') if tag.string]

sample_dict_1 = extract_info(tags)
sample_dict_1

../data/template_2018.hwpx


{'보고일': '2001-11-01',
 '보고자': '곽길동',
 '평가일': '2002-01-06',
 '평가자': '공길동',
 '병록번호': '11102223',
 '환자명': '홍길동',
 '나이': '11',
 '성별': '여성',
 '유해사례 보고 내용': '식은땀, 설사, 어지러움',
 '약사평가': '국가는 주택개발정책등을 통하여 모든 국민이 쾌적한 주거생활을 할 수 있도록 노력하여야 한다. 국회는 상호원조 또는 안전보장에 관한 조약, 중요한 국제조직에 관한 조약, 우호통상항해조약, 주권의 제약에 관한 조약, 강화조약, 국가나 국민에게 중대한 재정적 부담을 지우는 조약 또는 입법사항에 관한 조약의 체결·비준에 대한 동의권을 가진다. 저작자·발명가·과학기술자와 예술가의 권리는 법률로써 보호한다.모든 국민은 보건에 관하여 국가의 보호를 받는다. 공개하지 아니한 회의내용의 공표에 관하여는 법률이 정하는 바에 의한다. 대한민국은 민주공화국이다. 재판의 전심절차로서 행정심판을 할 수 있다. 행정심판의 절차는 법률로 정하되, 사법절차가 준용되어야 한다.',
 'WHO-UMC causality assessment': 'probable',
 'Naranjo Scale': 'probable',
 '한국형 Algorithm Ver 2.0': 'probable',
 '한국형 Algorithm Ver 2.0 (Total Score)': '9',
 'Naranjo Scale (Total Score)': '6'}

In [47]:
# ../data/template_2014.hwpx

def remove_special_characters(text):
    pattern = r'[^a-zA-Z0-9가-힣]'
    return re.sub(pattern, '', text)

def preprocess_tag_string(tag):
    _preprocessing_tag_string = re.sub(r'^\s*:\s*', '', tag.string)
    _preprocessing_tag_string = re.sub(r'^\s*-\s*', '', _preprocessing_tag_string)
    return _preprocessing_tag_string.strip()

def extract_info(arr_tag_string):
    _dict = {}
    keys = _info_dict.keys()
    length = len(arr_tag_string)
    i = 0
    value = ''
    while i < length:
        # print(arr_tag_string[i])
        try:
            if arr_tag_string[i] in keys:
                target_key = arr_tag_string[i]
                value = ''
            
            elif arr_tag_string[i] == '담당의사 등 전문가 의견' or arr_tag_string[i] == '환자정보':
                pass
            
            elif arr_tag_string[i] == 'WHO-UMC causality assessment':
                _dict['WHO-UMC causality assessment'] = arr_tag_string[i+1]
                
            
            elif arr_tag_string[i] == 'Naranjo Scale':
                _dict['Naranjo Scale'] = arr_tag_string[i+7]
                _dict['Naranjo Scale (Total Score)'] = arr_tag_string[i+6]
                    
            
            elif arr_tag_string[i] == '한국형 Algorithm Ver 2.0':
                _dict['한국형 Algorithm Ver 2.0'] = arr_tag_string[i+3]
                _dict['한국형 Algorithm Ver 2.0 (Total Score)'] = arr_tag_string[i+2]
            
            else:
                value += arr_tag_string[i]
                
            if i == length -1 or arr_tag_string[i+1] in keys:
                _dict[target_key] = value
                
        except:
            pass
                
        i += 1
    del _dict['참고문헌']
    return _dict
            
keys = _info_dict.keys()

soup = arr_soup[6]
print(soup[0])
tags = [preprocess_tag_string(tag.string) for tag in soup[1].select('hp\:t') if tag.string]

sample_dict_2 = extract_info(tags)
sample_dict_2

../data/template_2014.hwpx


{'보고일': '2000-01-01',
 '보고자': '홍길동',
 '평가일': '20000102',
 '평가자': '김길동',
 '병록번호': '0000****',
 '환자명': '김**',
 '나이': '01',
 '성별': '여자',
 '유해사례 내용': '(신경정신) 통증, (근골격계) 근골격계 통증',
 '약사평가': '헌법재판소 재판관은 탄핵 또는 금고 이상의 형의 선고에 의하지 아니하고는 파면되지 아니한다. 모든 국민은 직업선택의 자유를 가진다. 국가원로자문회의의 의장은 직전대통령이 된다. 다만, 직전대통령이 없을 때에는 대통령이 지명한다. 공무원은 국민전체에 대한 봉사자이며, 국민에 대하여 책임을 진다. 국회의원은 국회에서 직무상 행한 발언과 표결에 관하여 국회외에서 책임을 지지 아니한다. 국회는 국가의 예산안을 심의·확정한다. 법률이 정하는 주요방위산업체에 종사하는 근로자의 단체행동권은 법률이 정하는 바에 의하여 이를 제한하거나 인정하지 아니할 수 있다. 대한민국은 통일을 지향하며, 자유민주적 기본질서에 입각한 평화적 통일 정책을 수립하고 이를 추진한다.',
 'WHO-UMC causality assessment': '가능함(Possible)',
 'Naranjo Scale': '관련이 있을 가능성이 있음(possible)',
 'Naranjo Scale (Total Score)': '3',
 '한국형 Algorithm Ver 2.0': '가능성 있음(Possible)',
 '한국형 Algorithm Ver 2.0 (Total Score)': '8'}

In [68]:
import pandas as pd



arr = [sample_dict_1, sample_dict_2]

df = pd.DataFrame(arr)
df['유해사례 보고 내용'] = df['유해사례 내용'].fillna(df['유해사례 보고 내용'])
del df['유해사례 내용']
df

,보고일,보고자,평가일,평가자,병록번호,환자명,나이,성별,유해사례 보고 내용,약사평가,WHO-UMC causality assessment,Naranjo Scale,한국형 Algorithm Ver 2.0,한국형 Algorithm Ver 2.0 (Total Score),Naranjo Scale (Total Score)
0,2001-11-01,곽길동,2002-01-06,공길동,11102223,홍길동,11,여성,"식은땀, 설사, 어지러움",국가는 주택개발정책등을 통하여 모든 국민이 쾌적한 주거생활을 할 수 있도록 노력하여...,probable,probable,probable,9,6
1,2000-01-01,홍길동,20000102,김길동,0000****,김**,01,여자,"(신경정신) 통증, (근골격계) 근골격계 통증",헌법재판소 재판관은 탄핵 또는 금고 이상의 형의 선고에 의하지 아니하고는 파면되지 ...,가능함(Possible),관련이 있을 가능성이 있음(possible),가능성 있음(Possible),8,3
